In [1]:
import torch
from torch import nn
import numpy as np
import torchsummary
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import torch.optim as optim
from thop import profile
import torchvision.datasets
import random
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([
    #transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to tensors
])
train_size = datasets.ImageFolder(root=r"D:\Projects\Weed Detection\resized_dataset\train", transform=transform)
val_size = datasets.ImageFolder(root=r"D:\Projects\Weed Detection\resized_dataset\valid", transform=transform)
test_size = datasets.ImageFolder(root=r"D:\Projects\Weed Detection\resized_dataset\test", transform=transform)
train_loader = DataLoader(train_size, batch_size=32, shuffle=True)
val_loader = DataLoader(val_size, batch_size=10, shuffle=False)
test_loader = DataLoader(val_size, batch_size=10, shuffle=False)
num_epochs = 20

best_val_loss = float('inf')
patience = 3
counter = 0

In [3]:
def flatten(w, k=3, s=1, p=0, m=True):
    """
    Returns the right size of the flattened tensor after
        convolutional transformation
    :param w: width of image
    :param k: kernel size
    :param s: stride
    :param p: padding
    :param m: max pooling (bool)
    :return: proper shape and params: use x * x * previous_out_channels

    Example:
    r = flatten(*flatten(*flatten(w=100, k=3, s=1, p=0, m=True)))[0]
    self.fc1 = nn.Linear(r*r*128, 1024)
    """
    return int((np.floor((w - k + 2 * p) / s) + 1) / (2 if m else 1)), k, s, p, m
r = flatten(*flatten(*flatten(w=100, k=3, s=1, p=0, m=True)))[0]
x= flatten(*flatten(*flatten(*flatten(224, 3, 1, 0, True))))[0]

In [3]:
class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=16, kernel_size=3, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=0)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0)
        # Add dropout layer for regularization (optional)
        self.dropout = nn.Dropout(p=0.25)
        self.flat = nn.Flatten()
        # Define fully-connected layers
        self.fc1 = nn.Linear(in_features=9216, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=num_classes)
        
    def forward(self, x):
        # Apply convolutional layers with activation and pooling
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(nn.functional.relu(self.conv4(x)))
        x = self.dropout(x)
        x = self.flat(x)
        # Apply fully-connected layers with activation
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Define hyperparameters (adjust as needed)
input_channels = 3  # Assuming RGB images
num_classes = 12  # Number of output classes

# Create an instance of the model
model = CNNModel(input_channels, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Define loss function and optimizer (choose appropriate ones for your task)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [5]:
summary = torchsummary.summary(model, input_size=(3, 224, 224))  # Replace with your input size
print(summary)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 222, 222]             448
         MaxPool2d-2         [-1, 16, 111, 111]               0
           Dropout-3         [-1, 16, 111, 111]               0
            Conv2d-4         [-1, 32, 109, 109]           4,640
         MaxPool2d-5           [-1, 32, 54, 54]               0
           Dropout-6           [-1, 32, 54, 54]               0
            Conv2d-7           [-1, 32, 52, 52]           9,248
         MaxPool2d-8           [-1, 32, 26, 26]               0
           Dropout-9           [-1, 32, 26, 26]               0
           Conv2d-10           [-1, 64, 24, 24]          18,496
        MaxPool2d-11           [-1, 64, 12, 12]               0
          Dropout-12           [-1, 64, 12, 12]               0
          Flatten-13                 [-1, 9216]               0
           Linear-14                   

In [7]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True) as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=100 * correct / total)
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    
    print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1

    if counter >= patience:
        print(f'Validation loss did not improve for {patience} consecutive epochs. Early stopping...')
        break

Epoch 1/20: 100%|██████████████████████████████████████| 1125/1125 [01:24<00:00, 13.28it/s, accuracy=80.7, loss=0.0174]


Validation Loss: 0.3578, Accuracy: 89.31%


Epoch 2/20: 100%|██████████████████████████████████████| 1125/1125 [01:23<00:00, 13.45it/s, accuracy=86.2, loss=0.0128]


Validation Loss: 0.3245, Accuracy: 88.72%


Epoch 3/20: 100%|█████████████████████████████████████| 1125/1125 [01:22<00:00, 13.60it/s, accuracy=89.3, loss=0.00992]


...

Validation Loss: 0.2105, Accuracy: 93.53%


Epoch 11/20: 100%|█████████████████████████████████████| 1125/1125 [01:23<00:00, 13.41it/s, accuracy=95.5, loss=0.0042]


Validation Loss: 0.1942, Accuracy: 94.06%


Epoch 12/20: 100%|████████████████████████████████████| 1125/1125 [01:23<00:00, 13.40it/s, accuracy=95.7, loss=0.00396]


Validation Loss: 0.1855, Accuracy: 94.81%
Validation loss did not improve for 3 consecutive epochs. Early stopping...


In [9]:
train_size.classes

['Common wheat',
 'Convolvulus arvensis',
 'Cotton',
 'Euphorbia peplus',
 'Grass',
 'Lolium multiflorum',
 'Maize',
 'Nutgrass',
 'Purslane',
 'Sesame',
 'Sugar beet',
 'Tomato']

In [12]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert images to tensors
])

class_labels = ['Common wheat',
 'Convolvulus arvensis',
 'Cotton',
 'Euphorbia peplus',
 'Grass',
 'Lolium multiflorum',
 'Maize',
 'Nutgrass',
 'Purslane',
 'Sesame',
 'Sugar beet',
 'Tomato']
# Load dataset
val_size = datasets.ImageFolder(root=r"D:\Projects\Weed Detection\balanced_dataset\test", transform=transform)

# Create data loader for validation set
val_loader = DataLoader(val_size, batch_size=10, shuffle=False)

model = CNNModel(input_channels, num_classes)

# Define loss function and optimizer (choose appropriate ones for your task)
optimizer = torch.optim.Adam(model.parameters())
# Replace the classifier of the EfficientNet model with a new one matching the number of classes
num_classes = len(class_labels)
# num_ftrs = model._fc.in_features
# model._fc = nn.Linear(num_ftrs, num_classes)

# Load trained model
model.load_state_dict(torch.load('best_model.pth'))

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Evaluate the model on the validation dataset
model.eval()
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Calculate accuracy for each class
class_accuracy = [class_correct[i] / class_total[i] if class_total[i] != 0 else 0 for i in range(num_classes)]
for i in range(num_classes):
    print(f"Accuracy for class {class_labels[i]}: {class_accuracy[i] * 100:.2f}%")

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 391/391 [01:02<00:00,  6.26it/s]

Accuracy for class Common wheat: 92.50%
Accuracy for class Convolvulus arvensis: 98.88%
Accuracy for class Cotton: 91.85%
Accuracy for class Euphorbia peplus: 99.28%
Accuracy for class Grass: 100.00%
Accuracy for class Lolium multiflorum: 88.66%
Accuracy for class Maize: 70.78%
Accuracy for class Nutgrass: 81.21%
Accuracy for class Purslane: 68.46%
Accuracy for class Sesame: 94.02%
Accuracy for class Sugar beet: 89.21%
Accuracy for class Tomato: 93.62%


In [42]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the image
image_path = r"C:\Users\mohab\Desktop\New folder\tomato\WhatsApp Image 2024-04-12 at 15.25.13_295d1696.jpg"
image = Image.open(image_path)

# Apply transformations
input_image = transform(image).unsqueeze(0)  # Add a batch dimension

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_image = input_image.to(device)
model = model.to(device)

# Get predictions
with torch.no_grad():
    output = model(input_image)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()

class_labels = ['Common wheat',
 'Convolvulus arvensis',
 'Cotton',
 'Euphorbia peplus',
 'Grass',
 'Lolium multiflorum',
 'Maize',
 'Nutgrass',
 'Purslane',
 'Sesame',
 'Sugar beet',
 'Tomato']

# Print prediction
print("Predicted Class:", class_labels[predicted_class])
print("Probability:", probabilities[predicted_class].item())

Predicted Class: Sesame
Probability: 0.888537585735321


In [43]:
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Dropout(0.25),
    nn.Linear(in_features=9216, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=num_classes),
)

In [45]:
model.to('cuda')

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Dropout(p=0.25, inplace=False)
  (14): Linear(in_features=9216, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=12, bias=True)
)